In [17]:
import random
import numpy as np

## Gerar individos e pupulação!

In [18]:
class Individo:
   def __init__(self, tamanho_tabuleiro, quanti_rainhas): #Construtor da classe
      self.tamanho_tabuleiro = tamanho_tabuleiro
      self.qunti_rainhas = quanti_rainhas
   def gera_individo(self): # Um individo é uma maneira de como as rainhas vão fica no tabuleiro!
      individo = []
      while len(individo) < self.qunti_rainhas: # Verifica se o quantiade de rainhas no tabuleiro
         xy = [] #Coordenada X e Y de uma rainha no tabuleiro
         xy.append(random.randint(0, self.tamanho_tabuleiro -1)) # Coordenada X aleatoria
         xy.append(random.randint(0, self.tamanho_tabuleiro -1)) # Coordenada Y aleatoria
         if xy in individo: # Verifica se a coordenada gerada já existe no tabuleiro 
            print("Individo já existente!")
         else:
            individo.append(xy)
      return individo # Retorna uma lista das posições das rainhas no tabuleiro

class Population: #Gera populção inicial com varios individos (Possiveis Solucões)
   def __init__(self, tam_pop): #Constratutor de classe
      self.tam_pop = tam_pop
      self.pop = []
   def gera_pop(self, tamanho_tabuleiro, quanti_rainhas):
      individo = Individo(tamanho_tabuleiro, quanti_rainhas) # Instacinado a classe Individo
      for i in range(self.tam_pop): # Gera a quantidade da pupulação
         self.pop.append(individo.gera_individo()) # Chamando o metdo de gerar individo
      return self.pop  #Retorna populçaõ inicial

## Fução fitness (Culcula qunatas rainhas estão se cruzando (Diagonais e linhas X Colunas))!

In [19]:
class Custos:

    def __init__(self, lista):
       self.lista = lista
       #self.lista_temp = []

    def calculo_custo(self):
      cont = 0
      lista_verificada = []
      def calculo_custo_LC():
         lista_x = []
         lista_y = []
         cont = 0
         for i in range(len(self.lista)):
            pop = self.lista[i]
            lista_x.append(pop[0])
            lista_y.append(pop[1])
         for j in range(0,len(lista_x)):
            soma_x = lista_x.count(j)
            soma_y = lista_y.count(j)
            if soma_x == 0 or soma_x == 1:
               soma_x = 0
            if soma_y == 0 or soma_y == 1:
               soma_y = 0
            if soma_x > 0:
               soma_x = soma_x-1
            if soma_y > 0:
               soma_y = soma_y-1
            cont = cont+(soma_x+soma_y) 
         return cont
          
      custoLC = calculo_custo_LC()
      
      def verifica_lc(lista1, lista2):
        verifica = False
        if lista1[0] == lista2[0] or lista1[1] == lista2[1]:
           verifica = True
        return verifica
        
      ## Calculando o custo das diagonais
      for i in range(0, len(self.lista)):
         lista_temp =  self.lista[:]        
         posi = self.lista[i]
         lista_temp.pop(i)
         for j in range(0, len(lista_temp)):
            lista1 = []
            
            verifica = lista_temp[j]
            
            lista1.append(verifica)
            lista1.append(posi)
            
            verfica_colunas = verifica_lc(posi, verifica)
            
            if lista1 in lista_verificada or verfica_colunas == True:
                print("já foi verificado / ta na mesma linha ou coluna!")

            elif ( posi[0] + posi[1]) == (verifica[0] + verifica[1])  or (posi[0] - posi[1]) == (verifica[0] - verifica[1]) :
                lista2 = []
                lista2.append(posi)
                lista2.append(verifica)
                lista_verificada.append(lista2)              
                cont+=1       
      return  custoLC, cont,custoLC+cont, lista_verificada 

In [ ]:
class Generation:
    def __init__(self, quant_gera, pop):
        self.quant_gera = quant_gera
        self.pop = pop


In [20]:
teste = Population(10)
teste2 = teste.gera_pop(8,8)
print(teste2[0])


Individo já existente!
Individo já existente!
Individo já existente!
Individo já existente!
Individo já existente!
Individo já existente!
[[1, 7], [5, 1], [0, 5], [1, 4], [7, 0], [3, 5], [2, 1], [5, 5]]
